In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.7 MB/s eta 0:00:00


In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler  # For data normalization
from sklearn.decomposition import PCA  # For dimensionality reduction
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN  # For clustering
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
import pickle
# from groq import Groq
from sklearn.metrics import silhouette_score
import pandas as pd
# from fastapi import FastAPI

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mathchi/diabetes-data-set")

print("Path to dataset files:", path)

100%|██████████| 8.91k/8.91k [00:00<00:00, 13.1MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/mathchi/diabetes-data-set/versions/1


In [3]:
import os

# List files in the downloaded dataset directory
files = os.listdir(path)
print("Dataset Files:", files)


Dataset Files: ['diabetes.csv']


In [4]:
import pandas as pd

# Load the dataset
file_path = os.path.join(path, "diabetes.csv")  # Replace with the actual file name
df = pd.read_csv(file_path)

# Display the first few rows
df.head()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:

# Fill missing values
columns_to_fix = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
df[columns_to_fix] = df[columns_to_fix].replace(0, np.nan)
df.fillna(df.mean(), inplace=True)

# Standardize the data (exclude Outcome column)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df.drop(columns=["Outcome"]))


In [6]:
from sklearn.neighbors import KNeighborsClassifier

# Step 1: Perform K-Means Clustering first (since KNN needs predefined clusters)
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
df["KMeans_Cluster"] = kmeans.fit_predict(scaled_data)

# Step 2: Train KNN on K-Means clusters
knn = KNeighborsClassifier(n_neighbors=5)  # Choose appropriate k
knn.fit(scaled_data, df["KMeans_Cluster"])
df["KNN_Cluster"] = knn.predict(scaled_data)  # Predict clusters using KNN

# Agglomerative Clustering
agglo = AgglomerativeClustering(n_clusters=3)
df["Agglo_Cluster"] = agglo.fit_predict(scaled_data)

# DBSCAN Clustering
dbscan = DBSCAN(eps=1.5, min_samples=5)
df["DBSCAN_Cluster"] = dbscan.fit_predict(scaled_data)

print("✅ Clustering using K-Means, Agglomerative, DBSCAN, and KNN completed!")


✅ Clustering using K-Means, Agglomerative, DBSCAN, and KNN completed!


In [7]:
# Function to compute silhouette score
def compute_silhouette(data, labels):
    return silhouette_score(data, labels) if len(set(labels)) > 1 else "N/A"

print(f"KMeans Silhouette Score: {compute_silhouette(scaled_data, df['KMeans_Cluster'])}")
print(f"Agglomerative Clustering Silhouette Score: {compute_silhouette(scaled_data, df['Agglo_Cluster'])}")
print(f"DBSCAN Silhouette Score: {compute_silhouette(scaled_data, df['DBSCAN_Cluster'])}")
print(f"KNN Silhouette Score: {compute_silhouette(scaled_data, df['KNN_Cluster'])}")

KMeans Silhouette Score: 0.19495826630875748
Agglomerative Clustering Silhouette Score: 0.11456100982050367
DBSCAN Silhouette Score: 0.01340865981190432
KNN Silhouette Score: 0.19395558838273996
